# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import gmaps
import pandas as pd
import numpy as np
import requests


# Google developer API key
from api_keys import g_key
# Configure gmaps
gmaps.configure(api_key=g_key)



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file_to_load= "Resources/output_file.csv"

# Read Purchasing File and store into Pandas data frame
weather_df = pd.read_csv(file_to_load)
weather_df.dropna(inplace = True) 
weather_df.head()

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Windspeed,Country,Date
0,Kanlagay,5.8757,121.2915,26.68,77,100,2.75,PH,1632112873
1,Mataura,-46.1927,168.8643,12.97,65,70,3.18,NZ,1632112541
2,Nikolskoye,59.7035,30.7861,4.94,77,100,4.23,RU,1632112673
3,Sørland,67.6670,12.6934,10.04,73,100,7.25,NO,1632112875
4,Mahébourg,-20.4081,57.7000,22.16,68,40,4.63,MU,1632112566


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:

# Store latitude and longitude in locations
locations = weather_df[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
humidity = weather_df["Humidity"]

In [5]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=300, point_radius=5)
                                 

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
filtered_df = weather_df[(weather_df['Temperature'] >21.11) & (weather_df['Temperature'] <=26.67)].dropna()
filtered_df = weather_df[(weather_df['Windspeed'] <10)].dropna()
filtered_df = weather_df[(weather_df['Cloudiness'] ==0)].dropna()
filtered_df


,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Windspeed,Country,Date
13,Taltal,-25.4000,-70.4833,14.81,76,0,1.64,CL,1632112886
14,Bredasdorp,-34.5322,20.0403,12.62,71,0,3.60,ZA,1632112623
15,Port Elizabeth,-33.9180,25.5701,14.16,77,0,5.14,ZA,1632112616
22,Punta Arenas,-53.1500,-70.9167,5.06,70,0,4.12,CL,1632112658
38,Busselton,-33.6500,115.3333,16.69,47,0,4.06,AU,1632112569
...,...,...,...,...,...,...,...,...,...
564,Aswān,24.0934,32.9070,25.61,27,0,3.09,EG,1632113562
568,Bagdarin,54.4333,113.6000,15.90,58,0,1.61,RU,1632113594
569,Southbridge,42.0751,-72.0334,11.75,95,0,0.43,US,1632113596
574,Twentynine Palms,34.1356,-116.0542,25.61,24,0,2.24,US,1632113600


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
hotel_df = filtered_df[["City", "Country", "Latitude", "Longitude"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
13,Taltal,CL,-25.4000,-70.4833,
14,Bredasdorp,ZA,-34.5322,20.0403,
15,Port Elizabeth,ZA,-33.9180,25.5701,
22,Punta Arenas,CL,-53.1500,-70.9167,
38,Busselton,AU,-33.6500,115.3333,
...,...,...,...,...,...
564,Aswān,EG,24.0934,32.9070,
568,Bagdarin,RU,54.4333,113.6000,
569,Southbridge,US,42.0751,-72.0334,
574,Twentynine Palms,US,34.1356,-116.0542,


In [13]:
# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "lodging",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address=requests.get(base_url, params=params)
    name_address=name_address.json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, 'Hotel Name'] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
13,Taltal,CL,-25.4000,-70.4833,Residencial Karime
14,Bredasdorp,ZA,-34.5322,20.0403,Bredasdorp Country Manor
15,Port Elizabeth,ZA,-33.9180,25.5701,"Radisson Blu Hotel, Port Elizabeth"
22,Punta Arenas,CL,-53.1500,-70.9167,Hotel Dreams Del Estrecho
38,Busselton,AU,-33.6500,115.3333,Pullman Bunker Bay Resort Margaret River Region
...,...,...,...,...,...
564,Aswān,EG,24.0934,32.9070,Nuba Nile
568,Bagdarin,RU,54.4333,113.6000,Gostevoy Dom Terem
569,Southbridge,US,42.0751,-72.0334,Sturbridge Host Hotel & Conference Center
574,Twentynine Palms,US,34.1356,-116.0542,Holiday Inn Express & Suites Twentynine Palms-...


In [14]:

# Add marker layer ontop of heat map
# Display figure

# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]


In [15]:
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# Plot Heatma